#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2019


# Homework 2:   word2vec + SVM + Evaluation

### 100 points [6% of your final grade]

### Due: Tuesday, February 26, 2019 by 11:59pm

*Goals of this homework:* Understand word2vec-like term embeddings,  explore real-world challenges with SVM-based classifiers, understand and implement several evaluation metrics.

*Submission instructions (eCampus):* To submit your homework, rename this notebook as `UIN_hw2.ipynb`. For example, my homework submission would be something like `555001234_hw2.ipynb`. Submit this notebook via eCampus (look for the homework 2 assignment there). Your notebook should be completely self-contained, with the results visible in the notebook. We should not have to run any code from the command line, nor should we have to run your code within the notebook (though we reserve the right to do so). So please run all the cells for us, and then submit.

*Late submission policy:* For this homework, you may use as many late days as you like (up to the 5 total allotted to you).

*Collaboration policy:* You are expected to complete each homework independently. Your solution should be written by you without the direct aid or help of anyone else. However, we believe that collaboration and team work are important for facilitating learning, so we encourage you to discuss problems and general problem approaches (but not actual solutions) with your classmates. You may post on Piazza, search StackOverflow, etc. But if you do get help in this way, you must inform us by **filling out the Collaboration Declarations at the bottom of this notebook**. 

*Example: I found helpful code on stackoverflow at https://stackoverflow.com/questions/11764539/writing-fizzbuzz that helped me solve Problem 2.*

The basic rule is that no student should explicitly share a solution with another student (and thereby circumvent the basic learning process), but it is okay to share general approaches, directions, and so on. If you feel like you have an issue that needs clarification, feel free to contact either me or the TA.

# Part 1: Term embeddings + SVM (80 points)

### Dataset


For this homework, we will still play with Yelp reviews from the [Yelp Dataset Challenge](https://www.yelp.com/dataset_challenge). As in Homework 1, you'll see that each line corresponds to a review on a particular business. Each review has a unique "ID" and the text content is in the "review" field. Additionally, this time, we also offer you the "label". If `label=1`, it means that this review is `Food-relevant`. If `label=0`, it means that this review is `Food-irrelevant`. Similarly, we have already done some basic preprocessing on the reviews, so you can just tokenize each review using whitespace.

There are about 40,000 reviews in total, in which about 20,000 reviews are "Food-irrelevant". We split the review data into two sets. *review_train.json* is the training set. *review_test.json* is the testing set. 

In [326]:
# Please load the dataset
# Your code below
import json
reviewTrain = []
with open('hw2_data/review_train.json','r') as file :
    for line in file :
        reviewTrain.append(json.loads(line))
trainLen = len(reviewTrain)
# print((reviewTrain[0]))   type : dict 
reviewTest = []
with open('hw2_data/review_test.json','r') as file :
    for line in file :
        reviewTest.append(json.loads(line))
testLen = len(reviewTest)
print(reviewTest[0])

{'review': 'not a bad place to brush up on skills or just go to practice they need to organize their classes better with the rotations keep couples in the center and all guests in a circle instructors also need to ensure lessons are taught at the appropriate level do not overwhelm your new guests better than fat cat but nowhere near the level of fred astaire', 'id': 'INxgoY8zIyKAfneid4gCIw', 'label': 1}


###  Pre-trained term embeddings

To save your time, you can make use of  pre-trained term embeddings. In this homework, we are using one of the great pre-trained models from [GloVe](https://nlp.stanford.edu/projects/glove/) based on 2 billion tweets. GloVe is quite similar to word2vec. Unzip the *glove.6B.50d.txt.zip* file and run the code below. You will be able to load the term embeddings model, with which each word can be represented with a 50-dimension vector.

In [396]:
# reload the pre-trained term embeddings
import numpy as np
with open("hw2_data/glove.6B.50d.txt", "r") as lines:
    model = {line.split()[0]: np.array(list(map(float, line.split()[1:])))
           for line in lines}
# print((model['the']))

Now, you have a vector representation for each word. First, we use the simple (arithmetic) **mean** of these vectors of words in a review to represent the review. *Note: Just ignore those words which are not in the corpus of this pre-trained model.*

In [265]:
# Please figure out the vector representation for each review in the training data and testing data.
# Your code below
def getList(target,model) :
    res = []
    for review in target :
        a = np.zeros(50)
        reviewLen = len(review['review'])
        temp = review['review'].split()
        for word in temp :
            if (word in model) :
                a = a + model[word]
        a = a / reviewLen
        res.append(a)
    return res

XTrain = getList(reviewTrain, model)
XTest = getList(reviewTest, model)
# print(XTrain[0])
# print(XTest[0])

In [266]:
# X_train = np.empty([trainLen,50])
# i = 0
# while i < len(XTrain) :
#     X_train[i] = np.ndarray.tolist(XTrain[i])
#     i += 1
# # print(X_train[0])

# X_test = np.empty([testLen,50])
# i = 0
# while i < len(XTest) :
#     X_test[i] = np.ndarray.tolist(XTest[i])
#     i += 1
# # print(X_test[0])

Y_train = []
for review in reviewTrain:
    Y_train.append(review['label'])

Y_test = []
for review in reviewTest:
    Y_test.append(review['label'])
# print(Y_test[0:10])

[1, 1, 1, 0, 1, 1, 1, 1, 0, 1]


### SVM
With the vector representations you get for each review, please train an SVM model to predict whether a given review is food-relevant or not. **You do not need to implement any classifier from scratch. You may use scikit-learn's built-in capabilities.** You can only train your model with reviews in *review_train.json*.

In [305]:
# SVM model training
# Your code here
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
clf = svm.SVC(kernel = 'linear')
# clf = svm.SVC()
clf.fit(XTrain, Y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Your goal is to predict whether a given review is food-relevant or not. Please report the overall accuracy, precision and recall of your model on the **testing data**. You should **implement the functions for accuracy, precision, and recall**.

In [306]:
# print(sum(pred_clf))

# get the n number of 0 in Y_test
Y_test0 = 0
Y_test1 = 0
for num in Y_test :
    if num == 0 :
        Y_test0 += 1
    else :
        Y_test1 += 1
# print(Y_test0)   5975
# print(Y_test1)     5945

pre0Correct = 0
pre1Correct = 0
k = 0 
while k < len(Y_test) :
    if Y_test[k] == pred_clf[k] and Y_test[k] == 0 :
        pre0Correct += 1
    elif Y_test[k] == pred_clf[k] and Y_test[k] == 1 :
        pre1Correct += 1
    k += 1
# print(pre0Correct)  5146
# print(pre1Correct)  5553 

In [307]:
# Precision : 
precision = (pre0Correct + pre1Correct) / len(Y_test)
print('precision is %s '% precision)

# recall :
recall = (pre0Correct / Y_test0 + pre1Correct / Y_test1) / 2
print('recall is %s '% recall)

# f score:
f = 2 * precision * recall / (precision + recall)
print('score is %s '% f)

precision is 0.8975671140939597 
recall is 0.8976587336498094 
score is 0.8976129215339772 


In [308]:
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test, pred_clf))

0.897567114094


In [309]:
# this result is used to verify my function's correctness
pred_clf = clf.predict(XTest)
# print(pred_clf)
len(pred_clf)
print(classification_report(Y_test, pred_clf))
print(confusion_matrix(Y_test, pred_clf))

# A B         recall 0 = A / (A + B)
# C D         recall 1 = D / (D + C)

             precision    recall  f1-score   support

          0       0.93      0.86      0.89      5975
          1       0.87      0.93      0.90      5945

avg / total       0.90      0.90      0.90     11920

[[5146  829]
 [ 392 5553]]


### Document-based embeddings

Instead of taking the mean of term embeddings, you can directly train a **doc2vec** model for paragraph or document embeddings. You can refer to the paper [Distributed Representations of Sentences and Documents](https://arxiv.org/pdf/1405.4053v2.pdf) for more details. And in this homework, you can make use of the implementation in [gensim](https://radimrehurek.com/gensim/models/doc2vec.html).

Now, you need to:
* Train a doc2vec model based on all reviews you have (training + testing sets).
* Use the embeddings from your doc2vec model to represent each review and train a new SVM model.
* Report the overall accuracy, precision and recall of your model on the testing data.

In [274]:
# Train a doc2vec
# Your code here
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
newReviewTrain = []
for review in reviewTrain :
    newReviewTrain.append(review['review'].split())
for review in reviewTest :
    newReviewTrain.append(review['review'].split())
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(newReviewTrain)]
# print(documents[0:2])
max_epochs = 40
vec_size = 50
alpha = 0.025

model2 = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
model2.build_vocab(documents)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model2.train(documents,
                total_examples=model2.corpus_count,
                epochs=model2.iter)
    # decrease the learning rate
    model2.alpha -= 0.0002
    # fix the learning rate, no decay
    model2.min_alpha = model2.alpha
model2.save("doc2vec.model")
print("Model2 Saved")    
# model = Doc2Vec(documents, vector_size=50, min_count=2, epochs=40)

# model = Doc2Vec(size=vec_size,alpha=alpha, min_alpha=0.00025,min_count=1,dm =1)

/Users/jiangbohuai/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:580: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0


/Users/jiangbohuai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
Model2 Saved


In [301]:
def getList2(target, model2) :
    res = []
    for review in target :
        a = np.zeros(50)
        reviewLen = len(review['review'])
        temp = review['review'].split()
        for word in temp:
            a = a + model2[word]
        a = a / reviewLen
        res.append(a)
    return res
XTrain2 = getList2(reviewTrain, model2)
XTest2 = getList2(reviewTest, model2)

In [303]:
# Train a SVM
# Your code here
dbe = svm.SVC(kernel = 'linear')
dbe.fit(XTrain2, Y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [316]:
# Report the performance
# Your code here
pred_dbe = dbe.predict(XTest2)
print(classification_report(Y_test, pred_dbe))
print(confusion_matrix(Y_test, pred_dbe))
accuracy = accuracy_score(Y_test, pred_dbe)
accuracy

             precision    recall  f1-score   support

          0       0.93      0.88      0.91      5975
          1       0.89      0.93      0.91      5945

avg / total       0.91      0.91      0.91     11920

[[5283  692]
 [ 413 5532]]


0.90729865771812079

What do you observe? How different are your results for the term-based average approach vs. the doc2vec approach? Why do you think this is?

*provide a brief (1-2 paragraph) discussion based on these questions.*

### Can you do better?

Finally, see if you can do better than either the word- or doc- based embeddings approach for classification. You may explore new features, new classifiers, etc. Whatever you like. Just provide your code and a justification.

## First Try : Neural Network
## Result : Neural Network has 0.91124 accuracy, which is better than 0.9073 got above

In [408]:
#code here
from sklearn.neural_network import MLPClassifier
mlpc = MLPClassifier(hidden_layer_sizes = (11,11,11), max_iter = 500)
mlpc.fit(XTrain, Y_train)
pred_mlpc = mlpc.predict(XTest)

In [410]:
# print(classification_report(Y_test, pred_mlpc))
# print(confusion_matrix(Y_test, pred_mlpc))
accuracy_mlpc = accuracy_score(Y_test, pred_mlpc)
print('the accuracy is', accuracy_mlpc)

the accuracy is 0.911241610738


## Second Try : Random Forest Classifier : it does not work better

In [318]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 200)
rfc.fit(XTrain, Y_train)
pred_rfc = rfc.predict(X_test)

In [319]:
print(classification_report(Y_test, pred_rfc))
print(confusion_matrix(Y_test, pred_rfc))
accuracyRFC = accuracy_score(Y_test, pred_rfc)
accuracyRFC

             precision    recall  f1-score   support

          0       0.93      0.86      0.89      5975
          1       0.87      0.94      0.90      5945

avg / total       0.90      0.90      0.90     11920

[[5112  863]
 [ 380 5565]]


0.89572147651006706

# Part 2: NDCG (20 points)

You calculated the recall and precision in Part 1 and now you get a chance to implement NDCG. 

Assume that Amy searches for "food-relevant" reviews in the **testing set** on two search engines `A` and `B`. Since the ground-truth labels for the reviews are unknown to A and B, they need to make a prediction for each review and then return a ranked list of results based on their probabilities. The results from A are in *search_result_A.json*, and the results from B are in *search_result_B.json*. Each line contains the id of a review and its corresponding ranking.

You can check their labels in *review_test.json* while calculating the NDCG scores. If a review is "food-relevant", the relevance score is 1. Otherwise, the relevance score is 0.

In [393]:
# NDCG for search_result_A.json
# Your code here
import math
resultA = []
with open('hw2_data/search_result_A.json','r') as file :
    for line in file :
        resultA.append(json.loads(line))
resultB = []
with open('hw2_data/search_result_B.json','r') as file :
    for line in file :
        resultB.append(json.loads(line))
        
def getDCG(search_result) :
    DCG = 0
    id1 = search_result[0]['id']
    for review in reviewTest :
        if review['id'] == id1 :
            rel = review['label']
            search_result[0]['label'] = rel
            DCG += review['label']
    # print(DCG)
    i = 2
    while i <= len(search_result) :
        rankInfo = search_result[i - 1]
        for review in reviewTest :
            if review['id'] == rankInfo['id'] :
                rel = review['label']
                DCG += (rel / math.log(i,2))
        rankInfo['label'] = rel
        i += 1
    return DCG, search_result

def getIDCG(search_result) :
    search_result = sorted(search_result, key = lambda i : i['label'],reverse = True)
    IDCG = 1
    i = 2
    while i <= len(search_result) :
        rankInfo = search_result[i - 1]
        IDCG += rankInfo['label'] / math.log(i, 2)
        i += 1
    return IDCG

In [394]:
# print(getDCG(resultA).search_result)
# print(getIDCG(resultA))
DCG_A = getDCG(resultA)[0]
# print(DCG1)
resultA = getDCG(resultA)[1]
IDCG_A = getIDCG(resultA)
print('The NDCG of search engine A is %s' % (DCG_A / IDCG_A))

The NDCG of search engine A is 0.9689322925726681


In [395]:
# NDCG for search_result_B.json
# Your code here
DCG_B = getDCG(resultB)[0]
resultB = getDCG(resultB)[1]
IDCG_B = getIDCG(resultB)
print('The NDCG of search engine B is %s' % (DCG_B / IDCG_B))

The NDCG of search engine B is 0.9971789169738982


## Collaboration declarations

*If you collaborated with anyone (see Collaboration policy at the top of this homework), you can put your collaboration declarations here.*

In [320]:
# https://www.youtube.com/watch?v=0Lt9w-BxKFQ&t=1988s         neural network and Random Forest Classifier
# https://medium.com/@mishra.thedeepak/doc2vec-simple-implementation-example-df2afbbfbad5     gensim tutorial